# Load Data

In [ ]:
import pandas as pd
import time, datetime
import multiprocessing as mp

- create loop to iterate through 15 minute time stamps

In [ ]:
url_1 = "http://data.gdeltproject.org/gdeltv2/" # first part of gdelt data link
url_2 = "20210101000000" # date time 
url_3 = ".gkg.csv.zip" # file type

In [ ]:

pd.to_datetime(int(url_2), format='%Y%m%d%H%M%S') + datetime.timedelta(minutes = 15) # test adding 15 mins to timestamp

In [ ]:
datetime.datetime.now() # get current time

In [ ]:
#url="http://data.gdeltproject.org/gdeltv2/20220119023000.gkg.csv.zip"
url = url_1 + url_2 + url_3 # combine URL parts
sample_df=pd.read_csv(url, sep="\t", header=None, on_bad_lines="warn") # read in data

In [ ]:
sample_df.head() # display data

# Gather and Append 2021 Data

- create a function that reads in 2021 data and appends it to a master file

In [ ]:
def create_master():
    url_1 = "http://data.gdeltproject.org/gdeltv2/" # first part of gdelt data link
    url_2 = "20210116000000" # date time 
    url_3 = ".gkg.csv.zip" # file type
    url = url_1 + url_2 + url_3 # combine URL parts
    aggregate_df = pd.read_csv(url, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape') # read in data
    current_time = pd.to_datetime(int("20210201000000"), format='%Y%m%d%H%M%S') #datetime.datetime.now() # get current time
    date_time = pd.to_datetime(int(url_2), format='%Y%m%d%H%M%S') # convert int to datetime
    
    while date_time <= current_time:
        date_time += datetime.timedelta(minutes = 15) # test adding 15 mins to timestamp
        url_2 = date_time.strftime("%Y%m%d%H%M%S")
        url = url_1 + url_2 + url_3 # combine URL parts
        print(url)
        new_df = pd.read_csv(url, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape') # read in data
        aggregate_df = aggregate_df.append(new_df)
    
    return aggregate_df

In [ ]:
http://data.gdeltproject.org/gdeltv2/20210116000000.gkg.csv.zip

- 20210105104500: no encoding 
- 20210106074500: no encoding

In [ ]:
pd.read_csv('http://data.gdeltproject.org/gdeltv2/20210106074500.gkg.csv.zip', sep="\t", header=None, on_bad_lines="warn")

In [ ]:
tic = time.perf_counter()

agg_df = create_master()

toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

In [ ]:
agg_df.head()

In [ ]:
agg_df.tail()

In [ ]:
all_files = []
url_1 = "http://data.gdeltproject.org/gdeltv2/" # first part of gdelt data link
url_2 = "20210101000000" # date time 
url_3 = ".gkg.csv.zip" # file type
current_time = pd.to_datetime(int("20210102000000"), format='%Y%m%d%H%M%S')
date_time = pd.to_datetime(int(url_2), format='%Y%m%d%H%M%S')
url = url_1 + url_2 + url_3 # combine URL parts
all_files.append(url)

while date_time <= current_time:
        date_time += datetime.timedelta(minutes = 15) # test adding 15 mins to timestamp
        url_2 = date_time.strftime("%Y%m%d%H%M%S")
        url = url_1 + url_2 + url_3 # combine URL parts
        all_files.append(url)

In [ ]:
tic = time.perf_counter()

#######

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

#######

toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

In [ ]:
frame.head()

In [ ]:
frame.tail()

In [ ]:
def get_file_list(start, end):
    all_files = []
    url_1 = "http://data.gdeltproject.org/gdeltv2/" # first part of gdelt data link
    url_3 = ".gkg.csv.zip" # file type
    current_time = pd.to_datetime(int(end), format='%Y%m%d%H%M%S')
    date_time = pd.to_datetime(int(start), format='%Y%m%d%H%M%S')

    while date_time <= current_time:
        url_2 = date_time.strftime("%Y%m%d%H%M%S")
        url = url_1 + url_2 + url_3 # combine URL parts
        all_files.append(url)
        date_time += datetime.timedelta(minutes = 15) # test adding 15 mins to timestamp
        
    return all_files

In [ ]:
def load_all_files(file_list):
    # blank list for files
    list_df = []
    
    # loop through all file names and load
    for filename in file_list:
        df = pd.read_csv(filename, sep="\t", header=None, on_bad_lines="warn", encoding= 'unicode_escape')
        list_df.append(df)
    
    # combine list of dataframes
    frame = pd.concat(list_df, axis=0, ignore_index=True)
    
    return frame

In [ ]:
print("Number of processors: ", mp.cpu_count())

In [ ]:
all_files = get_file_list("20210101000000", "20210101001500")
all_files

In [ ]:
load_all_files(all_files)

In [ ]:
with mp.Pool(processes=core_count) as pool:
    df1= pool.map(load_all_files, all_files)

In [ ]:
core_count = mp.cpu_count()
all_files = get_file_list("20210101000000", "20210101001500")

tic = time.perf_counter()
#######

with mp.Pool(processes=core_count) as pool:
    df1= pool.map(load_all_files, all_files)
    
#######
toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

In [ ]:
tic = time.perf_counter()
#######

pool = mp.Pool(core_count)

df2 = pool.map(load_all_files, all_files) # data is the list

pool.close()
    
#######
toc = time.perf_counter()
print(f"Downloaded and appended in {(toc - tic)/60:0.2f} minutes")

In [ ]:
pool = mp.Pool(core_count)

df2 = pool.map(load_all_files, all_files) # data is the list

pool.close()